# Analyse the differences between control run and 10% increasing only in Mackenzie runoff

### Set-up and infrastructure

In [1]:
# tell Python to use the ecco_v4_py in the 'ECCOv4-py' repository
from os.path import join,expanduser
import sys

# identify user's home directory
user_home_dir = expanduser('~')

# import the ECCOv4 py library 
sys.path.insert(0,join(user_home_dir,'ECCOv4-py'))
import ecco_v4_py as ecco

import botocore  
import boto3
import os
import glob
import cmocean
import re
from boto3.session import Session
import cmocean
from collections import Counter
from dask.distributed import Client
import datetime
import fsspec
from gc import get_referents
import json
import numpy as np
from pathlib import Path
from pprint import pprint
import requests
import pandas as pd
import s3fs
import sys
from sys import getsizeof
import time as time
from types import ModuleType, FunctionType
import xarray as xr
import matplotlib.pyplot as plt
import zarr
import matplotlib.cm as cm
import imageio

In [2]:
# Use this for the netcdf files stored on an s3 bucket
def get_credentials(use_earthdata=False):
    """
    This routine automatically pulls your EDL crediential from .netrc file and use it to obtain an AWS S3 credential 
    through a PO.DAAC service accessible at https://archive.podaac.earthdata.nasa.gov/s3credentials.
    From the PO.DAAC Github (https://podaac.github.io/tutorials/external/July_2022_Earthdata_Webinar.html).
    
    Returns:
    =======
    
    credentials: a dictionary with AWS secret_key, access_key, and token
    """
    # NASA EarthData hosts ECCO V4r4 fileds
    if use_earthdata == False:
        session = boto3.Session()
        credentials_b3 = session.get_credentials()
        creds_b3 = credentials_b3.get_frozen_credentials()
        
        credentials = dict()
        credentials['secretAccessKey'] = credentials_b3.secret_key
        credentials['accessKeyId'] = credentials_b3.access_key
        credentials['sessionToken'] = credentials_b3.token

    # A 'public' AWS s3 bucket hosts V4r5 fields (they will eventually move to PO.DAAC)
    else:
        credentials = requests.get('https://archive.podaac.earthdata.nasa.gov/s3credentials').json()
    
    return credentials
    

In [3]:
def init_S3FileSystem(use_earthdata=False, requester_pays=True):
    """
    This routine automatically creates an 's3 file system' object and credentials dictionary.
    The s3 file system needs to be initialized with the special aws credentials.
    
    Returns:
    =======
    
    s3: an AWS S3 filesystem, 
    credentials: a dictionary with AWS secret_key, access_key, and token

    """
    credentials = get_credentials(use_earthdata=use_earthdata)

    if use_earthdata:
        requester_pays = False
        
    s3 = s3fs.S3FileSystem(requester_pays=requester_pays,
                           anon=False,
                           key=credentials['accessKeyId'],
                           secret=credentials['secretAccessKey'], 
                           token=credentials['sessionToken'])
    
    return s3, credentials

In [4]:
# function for determining the memory footprint of an object

# ... from https://stackoverflow.com/questions/449560/how-do-i-determine-the-size-of-an-object-in-python

# Custom objects know their class.
# Function objects seem to know way too much, including modules.
# Exclude modules as well.
BLACKLIST = type, ModuleType, FunctionType

def getsize(obj):
    """
    This routine returns the in-memory size of an python object
    
    Returns:
    =======
    
    size: size of object & members.
    """
    if isinstance(obj, BLACKLIST):
        raise TypeError('getsize() does not take argument of type: '+ str(type(obj)))
    seen_ids = set()
    size = 0
    objects = [obj]
    while objects:
        need_referents = []
        for obj in objects:
            if not isinstance(obj, BLACKLIST) and id(obj) not in seen_ids:
                seen_ids.add(id(obj))
                size += sys.getsizeof(obj)
                need_referents.append(obj)
        objects = get_referents(*need_referents)
    return size

In [5]:
from datetime import timedelta, datetime

def date_to_iter_number(date,seconds_per_iter = 3600):
    total_seconds = (date-datetime(1992,1,1)).total_seconds()
    iter_number = total_seconds/seconds_per_iter
    return(iter_number)

def iter_number_to_date(iter_number,seconds_per_iter=3600):
    total_seconds = iter_number*seconds_per_iter
    date = datetime(1992,1,1) + timedelta(seconds=total_seconds)
    return(date)

In [7]:
from dask.distributed import Client

#  connect to existing LocalCluster
# the port number will be different!
client = Client("tcp://127.0.0.1:41497")
client.ncores
client.restart()

In [8]:
# local path to monthly-mean native grid datasets
ecco_v4r5_mon_mean_native_dir = Path('/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/')

# list sub-directories (one per dataset)
ecco_v4r5_mon_mean_native_dataset_paths = np.sort(list(ecco_v4r5_mon_mean_native_dir.glob('*')))

for i, d in enumerate(ecco_v4r5_mon_mean_native_dataset_paths):
    print(str(i).zfill(3),d)

000 /efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc


In [ ]:
# ---------------------------------------------------------------
# Check november dates -- can change to your year as required
# ---------------------------------------------------------------
from datetime import timedelta, datetime

oct_start = date_to_iter_number(datetime(2014,10,1))
dec_end = date_to_iter_number(datetime(2014,12,31))

In [ ]:
# --------------------------------------------------
# Take the geometry file for the grid information
# --------------------------------------------------

geom = xr.open_dataset('/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc')

## Loading Salinity gifs for runs of tenpercent only in Mackenzie minus base

In [10]:
# -------------------------------------------------
# Load in three months of control run -- November
# ------------------------------------------------

input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean'
pattern = os.path.join(input_dir, 'SALT_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

salt_DA_list_base = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            salt_test = ecco.read_llc_to_tiles(input_dir, filename);
            salt_test = np.where(geom.hFacC == 1, salt_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            salt_DA = xr.DataArray(
                salt_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            salt_DA_list_base.append(salt_DA);

# Concatenate all valid files
salt_base_OND2014 = xr.concat(salt_DA_list_base, dim='time');

load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean/SALT_daily_mean.0000199416.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean/SALT_daily_mean.0000199440.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SALT_daily_mean/SALT_daily_mean.0000199464.data
load_binary_array: data a

In [11]:
# --------------------------------------------------------------------
# Load in three months of tenpercent in Mackenzie runoff -- November
# --------------------------------------------------------------------

input_dir = '/efs_ecco/cwilliam/tenpercent_mack/SALT_daily_mean'
pattern = os.path.join(input_dir, 'SALT_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

salt_DA_list_tenpercent_mack = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            salt_test = ecco.read_llc_to_tiles(input_dir, filename);
            salt_test = np.where(geom.hFacC == 1, salt_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            salt_DA = xr.DataArray(
                salt_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            salt_DA_list_tenpercent_mack.append(salt_DA);

# Concatenate all valid files
salt_tenpercent_mack_OND2014 = xr.concat(salt_DA_list_tenpercent_mack, dim='time');

load_binary_array: loading file /efs_ecco/cwilliam/tenpercent_mack/SALT_daily_mean/SALT_daily_mean.0000199416.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent_mack/SALT_daily_mean/SALT_daily_mean.0000199440.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent_mack/SALT_daily_mean/SALT_daily_mean.0000199464.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compa

In [11]:
# Assume salt_all is your DataArray: (time, j, i)
# If it's 4D (e.g., time, k, j, i), select a level:
data_baserun = salt_base_OND2014.isel(k=0, tile=6)
data_tenpercent_mackrun = salt_tenpercent_mack_OND2014.isel(k=0, tile=6)

# Directory for temporary frames
tmp_dir = 'tmp_frames'
os.makedirs(tmp_dir, exist_ok=True)

filenames = []

# Loop through time steps
for t in range(len(data_baserun.time)):  # or len(data.time) to use all
    fig, ax = plt.subplots(figsize=(6, 5))
    cmap = cm.get_cmap('bwr').copy()
    cmap.set_bad(color='gray')
    im = ax.imshow(data_tenpercent_mackrun.isel(time=t).values - data_baserun.isel(time=t).values, origin='lower', cmap=cmap, vmin=-0.5, vmax=0.5)
    ax.set_title(str(data_baserun.time[t].values))  # Use time label
    fig.colorbar(im, ax=ax)
    
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename)
    plt.close()
    filenames.append(filename)

# Create GIF
gif_filename = 'salt_tenper_mack_minus_base_OND2014.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Cleanup
for filename in filenames:
    os.remove(filename)
os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

/tmp/ipykernel_2206/798634717.py:18: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('bwr').copy()
/tmp/ipykernel_2206/798634717.py:33: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF saved to salt_base_minus_tenper_mack_OND2019.gif


## Loading Sea Ice Area gifs for runs of tenpercent only in Mackenzie minus base

In [12]:
# -------------------------------------------------
# Load in three months of control run -- November 
# -------------------------------------------------

input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIarea_daily_mean'
pattern = os.path.join(input_dir, 'SIarea_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

siarea_DA_list_base = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            siarea_test = ecco.read_llc_to_tiles(input_dir, filename);
            siarea_test = np.where(geom.hFacC == 1, siarea_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            siarea_DA = xr.DataArray(
                siarea_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            siarea_DA_list_base.append(siarea_DA);

# Concatenate all valid files
siarea_base_OND2014 = xr.concat(siarea_DA_list_base, dim='time');

load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIarea_daily_mean/SIarea_daily_mean.0000243240.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIarea_daily_mean/SIarea_daily_mean.0000243264.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIarea_daily_mean/SIarea_daily_mean.0000243288.data
load_binary_a

In [13]:
# --------------------------------------------------------------------
# Load in three months of tenpercent in Mackenzie runoff -- November
# --------------------------------------------------------------------

input_dir = '/efs_ecco/cwilliam/tenpercent_mack/SIarea_daily_mean'
pattern = os.path.join(input_dir, 'SIarea_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

siarea_DA_list_tenpercent_mack = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            siarea_test = ecco.read_llc_to_tiles(input_dir, filename);
            siarea_test = np.where(geom.hFacC == 1, siarea_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            siarea_DA = xr.DataArray(
                siarea_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            siarea_DA_list_tenpercent_mack.append(siarea_DA);

# Concatenate all valid files
siarea_tenpercent_mack_OND2014 = xr.concat(siarea_DA_list_tenpercent_mack, dim='time');

load_binary_array: loading file /efs_ecco/cwilliam/tenpercent_mack/SIarea_daily_mean/SIarea_daily_mean.0000243240.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent_mack/SIarea_daily_mean/SIarea_daily_mean.0000243264.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent_mack/SIarea_daily_mean/SIarea_daily_mean.0000243288.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >

In [14]:
# Assume salt_all is your DataArray: (time, j, i)
# If it's 4D (e.g., time, k, j, i), select a level:
data_baserun = siarea_base_OND2014.isel(k=0, tile=6)
data_tenpercent_mackrun = siarea_tenpercent_mack_OND2014.isel(k=0, tile=6)

# Directory for temporary frames
tmp_dir = 'tmp_frames'
os.makedirs(tmp_dir, exist_ok=True)

filenames = []

# Loop through time steps
for t in range(len(data_baserun.time)):  # or len(data.time) to use all
    fig, ax = plt.subplots(figsize=(6, 5))
    cmap = cm.get_cmap('PiYG').copy()
    cmap.set_bad(color='gray')
    im = ax.imshow(data_tenpercent_mackrun.isel(time=t).values - data_baserun.isel(time=t).values, origin='lower', cmap=cmap, vmin=-0.05, vmax=0.05)
    ax.set_title(str(data_baserun.time[t].values))  # Use time label
    fig.colorbar(im, ax=ax)
    
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename)
    plt.close()
    filenames.append(filename)

# Create GIF
gif_filename = 'siarea_tenper_mack_minus_base_OND2014.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Cleanup
for filename in filenames:
    os.remove(filename)
os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

/tmp/ipykernel_2206/1438960699.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('PiYG').copy()
/tmp/ipykernel_2206/1438960699.py:32: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF saved to siarea_base_minus_tenper_mack_OND2019.gif


## Loading Sea Ice Effective Thickness gifs for runs of tenpercent only in Mackenzie minus base

In [45]:
# -------------------------------------------------
# Load in three months of control run -- November 
# -------------------------------------------------

input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIheff_daily_mean'
pattern = os.path.join(input_dir, 'SIheff_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

siheff_DA_list_base = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            siheff_test = ecco.read_llc_to_tiles(input_dir, filename);
            siheff_test = np.where(geom.hFacC == 1, siheff_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            siheff_DA = xr.DataArray(
                siheff_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            siheff_DA_list_base.append(siheff_DA);

# Concatenate all valid files
siheff_base_OND2014 = xr.concat(siheff_DA_list_base, dim='time');

load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIheff_daily_mean/SIheff_daily_mean.0000199416.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIheff_daily_mean/SIheff_daily_mean.0000199440.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/SIheff_daily_mean/SIheff_daily_mean.0000199464.data
load_binary_a

In [46]:
# --------------------------------------------------------------------
# Load in three months of tenpercent in Mackenzie runoff -- November
# --------------------------------------------------------------------

input_dir = '/efs_ecco/cwilliam/tenpercent_mack/SIheff_daily_mean'
pattern = os.path.join(input_dir, 'SIheff_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

siheff_DA_list_tenpercent_mack = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            siheff_test = ecco.read_llc_to_tiles(input_dir, filename);
            siheff_test = np.where(geom.hFacC == 1, siheff_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            siheff_DA = xr.DataArray(
                siheff_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            siheff_DA_list_tenpercent_mack.append(siheff_DA);

# Concatenate all valid files
siheff_tenpercent_mack_OND2014 = xr.concat(siheff_DA_list_tenpercent_mack, dim='time');

load_binary_array: loading file /efs_ecco/cwilliam/tenpercent_mack/SIheff_daily_mean/SIheff_daily_mean.0000199416.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent_mack/SIheff_daily_mean/SIheff_daily_mean.0000199440.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent_mack/SIheff_daily_mean/SIheff_daily_mean.0000199464.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >

In [47]:
# Assume salt_all is your DataArray: (time, j, i)
# If it's 4D (e.g., time, k, j, i), select a level:
data_baserun = siheff_base_OND2014.isel(k=0, tile=6)
data_tenpercent_mackrun = siheff_tenpercent_mack_OND2014.isel(k=0, tile=6)

# Directory for temporary frames
tmp_dir = 'tmp_frames'
os.makedirs(tmp_dir, exist_ok=True)

filenames = []

# Loop through time steps
for t in range(len(data_baserun.time)):  # or len(data.time) to use all
    fig, ax = plt.subplots(figsize=(6, 5))
    cmap = cm.get_cmap('BrBG').copy()
    cmap.set_bad(color='gray')
    im = ax.imshow(data_tenpercent_mackrun.isel(time=t).values - data_baserun.isel(time=t).values, origin='lower', cmap=cmap, vmin=-0.1, vmax=0.1)
    ax.set_title(str(data_baserun.time[t].values))  # Use time label
    fig.colorbar(im, ax=ax)
    
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename)
    plt.close()
    filenames.append(filename)

# Create GIF
gif_filename = 'siheff_tenper_mack_minus_base_OND2014.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Cleanup
for filename in filenames:
    os.remove(filename)
os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

/tmp/ipykernel_470/14100831.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('BrBG').copy()
/tmp/ipykernel_470/14100831.py:32: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF saved to siheff_tenper_mack_minus_base_OND2014.gif


## Loading Potential Temperature gifs for runs of tenpercent only in Mackenzie minus base

In [48]:
# -------------------------------------------------
# Load in three months of control run -- November
# -------------------------------------------------

input_dir = '/efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/THETA_daily_mean'
pattern = os.path.join(input_dir, 'THETA_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

theta_DA_list_base = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            theta_test = ecco.read_llc_to_tiles(input_dir, filename);
            theta_test = np.where(geom.hFacC == 1, theta_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            theta_DA = xr.DataArray(
                theta_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            theta_DA_list_base.append(theta_DA);

# Concatenate all valid files
theta_base_OND2014 = xr.concat(theta_DA_list_base, dim='time');

load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/THETA_daily_mean/THETA_daily_mean.0000199416.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/THETA_daily_mean/THETA_daily_mean.0000199440.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/obousque/r5/WORKINGDIR/ECCOV4/release5/run/diags/THETA_daily_mean/THETA_daily_mean.0000199464.data
load_binary_array: 

In [49]:
# --------------------------------------------------------------------
# Load in three months of tenpercent in Mackenzie runoff -- November
# --------------------------------------------------------------------

input_dir = '/efs_ecco/cwilliam/tenpercent_mack/THETA_daily_mean'
pattern = os.path.join(input_dir, 'THETA_daily_mean.*.data')
file_list = sorted(glob.glob(pattern))

# Define your range
start_num = oct_start
end_num = dec_end

theta_DA_list_tenpercent_mack = []

for filepath in file_list:
    filename = os.path.basename(filepath);
    
    # Extract last 6 digits using regex
    match = re.search(r'(\d{6})\.data$', filename)
    if match:
        number = int(match.group(1))
        if start_num <= number <= end_num:
            theta_test = ecco.read_llc_to_tiles(input_dir, filename);
            theta_test = np.where(geom.hFacC == 1, theta_test, np.nan);

            tile = range(1, 14)
            i = range(90)
            j = range(90)
            k = range(50)
            time = iter_number_to_date(number)

            theta_DA = xr.DataArray(
                theta_test,
                coords={'time': time, 'k': k, 'tile': tile, 'j': j, 'i': i},
                dims=['k', 'tile', 'j', 'i']
            );

            theta_DA_list_tenpercent_mack.append(theta_DA);

# Concatenate all valid files
theta_tenpercent_mack_OND2014 = xr.concat(theta_DA_list_tenpercent_mack, dim='time');

load_binary_array: loading file /efs_ecco/cwilliam/tenpercent_mack/THETA_daily_mean/THETA_daily_mean.0000199416.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent_mack/THETA_daily_mean/THETA_daily_mean.0000199440.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
load_binary_array: loading file /efs_ecco/cwilliam/tenpercent_mack/THETA_daily_mean/THETA_daily_mean.0000199464.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc

In [50]:
# Assume salt_all is your DataArray: (time, j, i)
# If it's 4D (e.g., time, k, j, i), select a level:
data_baserun = theta_base_OND2014.isel(k=0, tile=6)
data_tenpercent_mackrun = theta_tenpercent_mack_OND2014.isel(k=0, tile=6)

# Directory for temporary frames
tmp_dir = 'tmp_frames'
os.makedirs(tmp_dir, exist_ok=True)

filenames = []

# Loop through time steps
for t in range(len(data_baserun.time)):  # or len(data.time) to use all
    fig, ax = plt.subplots(figsize=(6, 5))
    cmap = cm.get_cmap('RdBu').copy()
    cmap.set_bad(color='gray')
    im = ax.imshow(data_tenpercent_mackrun.isel(time=t).values - data_baserun.isel(time=t).values, origin='lower', cmap=cmap, vmin=-0.5, vmax=0.5)
    ax.set_title(str(data_baserun.time[t].values))  # Use time label
    fig.colorbar(im, ax=ax)
    
    filename = os.path.join(tmp_dir, f'frame_{t:03d}.png')
    plt.savefig(filename)
    plt.close()
    filenames.append(filename)

# Create GIF
gif_filename = 'theta_tenper_mack_minus_base_OND2014.gif'
with imageio.get_writer(gif_filename, mode='I', duration=0.3) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Cleanup
for filename in filenames:
    os.remove(filename)
os.rmdir(tmp_dir)

print(f"GIF saved to {gif_filename}")

/tmp/ipykernel_470/730840999.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('RdBu').copy()
/tmp/ipykernel_470/730840999.py:32: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF saved to theta_tenper_mack_minus_base_OND2014.gif
